In [5]:
!pip install haversine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
from sklearn.cluster import KMeans
import json
import psycopg2
from datetime import datetime, date
import math
import numpy as np
from haversine import haversine

In [7]:
def load_db():
  with open("/content/drive/MyDrive/yeoreodigm/data_files/db_info.json") as json_file:
    DB_INFO = json.load(json_file)
  
  endpoint = DB_INFO["ENDPOINT"]
  dbname = DB_INFO["DB_NAME"]
  user = DB_INFO["USER_ID"]
  password = DB_INFO["PASSWORD"]
  db = psycopg2.connect(host=endpoint,dbname=dbname,user=user,password=password)
  return db

In [8]:
def load_places_location(db, place_list):
  cursor = db.cursor()
  place_list = str(tuple(place_list))
  sql = f"select place_id, latitude, longitude from places where place_id in {place_list}"
  cursor.execute(sql)
  result = cursor.fetchall()
  result = list(map(list,result))
  return result

In [10]:
def calculate_distance_haversine(A,B):
  A_x = A[1]
  A_y = A[2]
  B_x = B[1]
  B_y = B[2]
  dist = haversine((A_x,A_y),(B_x,B_y))
  return dist

In [11]:
db = load_db()

In [12]:
load_places_location(db,[7,3,9,2])

[[2, Decimal('33.5194929'), Decimal('126.9510302')],
 [3, Decimal('33.4621574'), Decimal('126.9363164')],
 [7, Decimal('33.3673209'), Decimal('126.3570070')],
 [9, Decimal('33.4302205'), Decimal('126.9280527')]]

In [13]:
def make_dist_matrix_haversine(location_list):
  matrix_size = len(location_list)
  #initialize empty n*n matrix
  dist_matrix = [([0] * matrix_size) for _ in range(matrix_size)]
  for i in range(matrix_size):
    for j in range(i+1,matrix_size):
      dist_matrix[i][j] = calculate_distance_haversine(location_list[i],location_list[j])
      dist_matrix[j][i] = dist_matrix[i][j]
  return dist_matrix

In [14]:
place_list = [1,2,3,4,5,6,7,8,9,10]
place_info = load_places_location(db,place_list)
dist_matrix = make_dist_matrix_haversine(place_info)

In [15]:
for i in dist_matrix:
  for j in i:
    print(format(j,"2.0f"),end=' ')
  print()

 0 31 28 18 20 18 27 37 27 29 
31  0  7 13 47 17 58 67 10 60 
28  7  0 12 43 17 55 65  4 56 
18 13 12  0 36  5 44 54 13 47 
20 47 43 36  0 37 23 34 40 18 
18 17 17  5 37  0 42 52 18 46 
27 58 55 44 23 42  0 11 53  9 
37 67 65 54 34 52 11  0 64 17 
27 10  4 13 40 18 53 64  0 54 
29 60 56 47 18 46  9 17 54  0 


In [16]:
def find_optimal(dist_matrix,day):
  AIRPORT = (33.5059364,126.4959513)
  print("dist matrix : ")
  for i in dist_matrix:
    for j in i:
      print(format(j,"2.0f"),end=' ')
    print()
  print("*"*30)
  total_dist = [0]*day
  visit_result = [[] for _ in range(day)]
  for i in range(day):
    start = i
    visit = [False] * day
    visit[i] = True    
    now = i
    min_dist = 100
    min_idx = 999
    visit_result[i].append(now)
    print("start : ",start)
    while False in visit: #모든 클러스터를 방문할때까지
      for idx, dist in enumerate(dist_matrix[now]):
        if visit[idx] == True:
          continue
        else:
          if dist < min_dist:
            min_idx = idx
            min_dist = dist
      print("from %d to %d, %.2f  "%(now,min_idx,min_dist))
      
      total_dist[i] += min_dist
      now = min_idx
      visit[now] = True
      visit_result[i].append(now)
      min_dist=999
    
    print("total_dist : ",total_dist[i])

  shortest_cluster = np.array(total_dist).argsort()[0]
  return visit_result[shortest_cluster]

In [17]:
optimal_path = find_optimal(dist_matrix,10)

dist matrix : 
 0 31 28 18 20 18 27 37 27 29 
31  0  7 13 47 17 58 67 10 60 
28  7  0 12 43 17 55 65  4 56 
18 13 12  0 36  5 44 54 13 47 
20 47 43 36  0 37 23 34 40 18 
18 17 17  5 37  0 42 52 18 46 
27 58 55 44 23 42  0 11 53  9 
37 67 65 54 34 52 11  0 64 17 
27 10  4 13 40 18 53 64  0 54 
29 60 56 47 18 46  9 17 54  0 
******************************
start :  0
from 0 to 5, 17.98  
from 5 to 3, 5.49  
from 3 to 2, 12.05  
from 2 to 8, 3.63  
from 8 to 1, 10.15  
from 1 to 4, 47.48  
from 4 to 9, 18.48  
from 9 to 6, 8.65  
from 6 to 7, 11.31  
total_dist :  135.22833527343286
start :  1
from 1 to 2, 6.52  
from 2 to 8, 3.63  
from 8 to 3, 12.81  
from 3 to 5, 5.49  
from 5 to 0, 17.98  
from 0 to 4, 19.70  
from 4 to 9, 18.48  
from 9 to 6, 8.65  
from 6 to 7, 11.31  
total_dist :  104.56535841909738
start :  2
from 2 to 8, 3.63  
from 8 to 1, 10.15  
from 1 to 3, 13.32  
from 3 to 5, 5.49  
from 5 to 0, 17.98  
from 0 to 4, 19.70  
from 4 to 9, 18.48  
from 9 to 6, 8.65  
from 6 to

In [18]:
print("%.2f,%f"%(3.5,4.5))

3.50,4.500000


In [19]:
optimal_path

[1, 2, 8, 3, 5, 0, 4, 9, 6, 7]

In [20]:
def make_dist_matrix_haversine(location_info):
  print("making matrix!!!")
  matrix_size = len(location_info)
  #initialize empty n*n matrix
  dist_matrix = [([0] * matrix_size) for _ in range(matrix_size)]
  for i in range(matrix_size):
    for j in range(i+1,matrix_size):
      dist_matrix[i][j] = calculate_distance_haversine(location_info[i],location_info[j])
      dist_matrix[j][i] = dist_matrix[i][j]
  return dist_matrix

In [21]:
from itertools import combinations

In [22]:
AIRPORT = (33.5059364,126.4959513)
haversine(place_info[0][1:],AIRPORT)

17.43496801230598

In [39]:

def optimize_course(place_list,day,db):
  place_info = load_places_location(db, place_list)
  save_id = []  
  result = [[] for _ in range(day)]
  
  AIRPORT = (33.5059364,126.4959513)
  length = len(place_list)
  total_dist = [0]*length
  visit_result = [[] for _ in range(length)]

  dist_matrix = make_dist_matrix_haversine(place_info)

  for i in range(length):
    start = i
    end = 999
    visit = [False] * length
    visit[i] = True    
    now = i
    min_dist = 100
    min_idx = 999
    visit_result[i].append(now)
    print("start : ",start)
    while False in visit: #모든 클러스터를 방문할때까지
      for idx, dist in enumerate(dist_matrix[now]):
        if visit[idx] == True:
          continue
        else:
          if dist < min_dist:
            min_idx = idx
            min_dist = dist
      print("dist from %d to %d : %.2f  "%(now,min_idx,min_dist))
      
      total_dist[start] += min_dist
      now = min_idx
      visit[now] = True
      visit_result[start].append(now)
      min_dist=999
      end = min_idx

    airport_to_start = haversine(AIRPORT,place_info[start][1:]) # 공항에서 시작지점 까지 거리
    end_to_airport = haversine(AIRPORT,place_info[end][1:]) #마지막지점에서 공항 가는 거리
    total_dist[i] += airport_to_start 
    total_dist[i] += end_to_airport  

    print("airport - 시작지점(%s) 까지 거리%s 합산"%(start,airport_to_start))
    print("끝지점(%s) - 공항  까지 거리%s 합산"%(end,end_to_airport))
    print("total_dist : ",total_dist[i])

  shortest_cluster = np.array(total_dist).argsort()[0]
  result_index = visit_result[shortest_cluster]  #최단거리를 보장하는 index 방문 순서 마지막에 이 인덱스랑 Id랑 매핑해줘야함.
 

  #day dividing ...
  day_dividing = []
  print("result_index[-1]: ",result_index[-1])
  for comb in combinations(result_index,day-1):
    #comb : (5,3,7)
    #print("comb : ",comb)
    if comb[-1] == result_index[-1]: 
      continue
    dist = 0
    for now in comb:
      next = result_index[result_index.index(now) + 1]
      now_loc = place_info[now][1:]
      next_loc = place_info[next][1:]
      
      dist += haversine(now_loc,next_loc)
    day_dividing.append((dist,comb))
    
  day_dividing.sort()
  divider = day_dividing[-1][1]
  print("divider : ",divider)
  final_result = []
  today_places = []
  start = 0
  for i in divider:
    cut = result_index.index(i)  
    final_result.append(result_index[start:cut+1])
    start = cut+1
  final_result.append(result_index[start:])
  #a = [list(map(float, suba)) for suba in a]
  #final_result = [( ) for places in final_result]
  final_result2 = []
  
  for i in final_result:
    tmp_list = []
    print("i : ",i)
    for j in i:
      print("j : ",j)
      tmp_list.append(place_info[j][0])
    final_result2.append(tmp_list)


  return final_result2



In [29]:
place_list = [9,7,3,5,1,6,12,31,18]
idx = load_places_location(db,place_list)


In [37]:
place_list.sort()

In [38]:
place_list

[1, 3, 5, 6, 7, 9, 12, 18, 31]

In [40]:
optimize_course(place_list,4,db)

making matrix!!!
start :  0
dist from 0 to 7 : 5.83  
dist from 7 to 6 : 9.83  
dist from 6 to 3 : 9.69  
dist from 3 to 1 : 16.95  
dist from 1 to 5 : 3.63  
dist from 5 to 2 : 39.97  
dist from 2 to 4 : 23.24  
dist from 4 to 8 : 10.37  
airport - 시작지점(0) 까지 거리17.43496801230598 합산
끝지점(8) - 공항  까지 거리18.786423002566767 합산
total_dist :  155.73167245618464
start :  1
dist from 1 to 5 : 3.63  
dist from 5 to 3 : 18.16  
dist from 3 to 6 : 9.69  
dist from 6 to 7 : 9.83  
dist from 7 to 0 : 5.83  
dist from 0 to 2 : 19.70  
dist from 2 to 4 : 23.24  
dist from 4 to 8 : 10.37  
airport - 시작지점(1) 까지 거리41.12900821221665 합산
끝지점(8) - 공항  까지 거리18.786423002566767 합산
total_dist :  160.35749408557786
start :  2
dist from 2 to 0 : 19.70  
dist from 0 to 7 : 5.83  
dist from 7 to 6 : 9.83  
dist from 6 to 3 : 9.69  
dist from 3 to 1 : 16.95  
dist from 1 to 5 : 3.63  
dist from 5 to 4 : 53.47  
dist from 4 to 8 : 10.37  
airport - 시작지점(2) 까지 거리29.64061814139395 합산
끝지점(8) - 공항  까지 거리18.786423002566767

[[1, 18, 12, 6], [3, 9], [5], [7, 31]]

###조합 확인해보기

In [ ]:
[[1, 8, 7, 4], [2, 6], [3], [5, 9]]
[[0, 7, 6, 3], [1, 5], [2], [4, 8]]

In [53]:
from itertools import combinations

In [73]:
place_info

[[1, Decimal('33.4077504'), Decimal('126.6424787')],
 [2, Decimal('33.5194929'), Decimal('126.9510302')],
 [3, Decimal('33.4621574'), Decimal('126.9363164')],
 [4, Decimal('33.4917178'), Decimal('126.8112799')],
 [5, Decimal('33.2447169'), Decimal('126.5595512')],
 [6, Decimal('33.5283774'), Decimal('126.7716157')],
 [7, Decimal('33.3673209'), Decimal('126.3570070')],
 [8, Decimal('33.3937480'), Decimal('126.2394319')],
 [9, Decimal('33.4302205'), Decimal('126.9280527')],
 [10, Decimal('33.2901402'), Decimal('126.3683652')]]

In [61]:
attractions = [5,17,2,67,13,1,97,654,13,254]
len(attractions)

10

In [93]:
attractions.index(67)

3

In [62]:
combinations(attractions,2)

In [63]:
cnt = 0
for i in combinations(attractions,2):
  cnt+=1
  print(i)

(5, 17)
(5, 2)
(5, 67)
(5, 13)
(5, 1)
(5, 97)
(5, 654)
(5, 13)
(5, 254)
(17, 2)
(17, 67)
(17, 13)
(17, 1)
(17, 97)
(17, 654)
(17, 13)
(17, 254)
(2, 67)
(2, 13)
(2, 1)
(2, 97)
(2, 654)
(2, 13)
(2, 254)
(67, 13)
(67, 1)
(67, 97)
(67, 654)
(67, 13)
(67, 254)
(13, 1)
(13, 97)
(13, 654)
(13, 13)
(13, 254)
(1, 97)
(1, 654)
(1, 13)
(1, 254)
(97, 654)
(97, 13)
(97, 254)
(654, 13)
(654, 254)
(13, 254)


###nested map으로 idx list - >  id 리스트로 변환해주기

In [127]:
test = [[8, 2, 1, 3, 5], [0], [4], [9, 6, 7]]

In [131]:
place_info[0][0]

1

In [134]:
def idx_to_id(x):
  return place_info[x][0]

  
list(map(idx_to_id,[8, 2, 1, 3, 5]))

[9, 3, 2, 4, 6]

In [138]:
for i in range(len(test)):
  test[i] = list(map(idx_to_id,test[i]))

In [139]:
test

[[9, 3, 2, 4, 6], [1], [5], [10, 7, 8]]

In [ ]:
def idx_to_id(x):
  return place_info[x][0]
for i in range(len(test)):
  test[i] = list(map(idx_to_id,test[i]))

###맵 2걔

In [2]:

a = [list(map(float, suba)) for suba in a]

TypeError: ignored